In [1]:
import pickle
from general_helpers import Helpers
from listar_planilhas import listar_planilhas_na_pasta
import os
from dotenv import load_dotenv

from googleapiclient.discovery import build
from google.oauth2.credentials import Credentials
import pandas as pd

from concurrent.futures import ThreadPoolExecutor, as_completed

helpers = Helpers()


In [2]:
helpers = Helpers()
load_dotenv()
LINK_GDRIVE = os.getenv('LINK_GDRIVE')
CREDENCIAIS_JSON = os.getenv('CREDENCIAIS_JSON')

pasta_id = helpers.extrair_id_pasta_drive(LINK_GDRIVE)

lista_ids = listar_planilhas_na_pasta(pasta_id=pasta_id, caminho_credenciais_json=CREDENCIAIS_JSON)

ids = [item['id'] for item in lista_ids]

len(ids)



✓ Encontradas 100 planilha(s) na pasta
[{'id': '1kWAiRKAxuxpE-31nHEUAQzUosnRpvMBQJKzO3lTdXXQ', 'name': 'tabela_ff605b5d796244d0b23ac91283f97aa4'}, {'id': '1dXIkPAfW7YirzYCVTZlB8JAG70jgjOfaDvTpVPgFRH4', 'name': 'tabela_fc7fa11e406b4ae8934a437c01a97031'}, {'id': '1OJTavWX9gDmzo0m6Dact5jy0SfDxHYt_PANcIdkjMqM', 'name': 'tabela_fc6d9d3d16cd42eda555edd5864dba98'}, {'id': '1NqbIT8JiCTBDTxmRem7x3j_otTLi99sq1Y3wvhiuPKc', 'name': 'tabela_f558437613b447819775dd42c50cdb5c'}, {'id': '1XA4-r_0QIEpjwnzX2_4PUzr-Pp0AhHHIHDBZYTkpTjI', 'name': 'tabela_f5501e77967e47b5b619193d0b617d5f'}]...]


100

In [13]:
def ler_sheets_por_ids(lista_ids):
    # Carrega as credenciais do pickle (binário!)
    with open("../token.pickle", "rb") as token:
        creds = pickle.load(token)

    service = build("sheets", "v4", credentials=creds)

    resultados = {}

    for i,sheet_id in enumerate(lista_ids, start=1):
        resp = service.spreadsheets().values().get(
            spreadsheetId=sheet_id,
            range="A:Z"
        ).execute()

        valores = resp.get("values", [])

        if valores:
            df = pd.DataFrame(valores[1:], columns=valores[0])
        else:
            df = pd.DataFrame()


        nome = f"sheet_{i}"
        # print(f"Lendo {nome}")
        df["origem"] = nome
        resultados[nome] = df

    return resultados

In [14]:
def ler_sheets_por_ids_com_esteroides(lista_ids, max_workers=5):
    
    def ler_uma_sheet(sheet_id, indice):
        """Cada thread cria seu próprio service"""
        # Carrega credenciais dentro da thread
        with open("../token.pickle", "rb") as token:
            creds = pickle.load(token)
        
        # Cria service exclusivo para esta thread
        service = build("sheets", "v4", credentials=creds)
        
        try:
            resp = service.spreadsheets().values().get(
                spreadsheetId=sheet_id,
                range="A:Z"
            ).execute()
            
            valores = resp.get("values", [])
            if valores:
                df = pd.DataFrame(valores[1:], columns=valores[0])
            else:
                df = pd.DataFrame()
            
            nome = f"sheet_{indice}"
            df["origem"] = nome
            
            return nome, df
            
        except Exception as e:
            print(f"Erro ao ler {sheet_id}: {e}")
            return f"sheet_{indice}", pd.DataFrame()
    
    # Executa em paralelo
    resultados = {}
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        # Enumera para manter os índices
        futures = {
            executor.submit(ler_uma_sheet, sid, i): (sid, i) 
            for i, sid in enumerate(lista_ids, start=1)
        }
        
        for future in as_completed(futures):
            nome, df = future.result()
            resultados[nome] = df
    
    return resultados

In [15]:
dict_result = ler_sheets_por_ids(lista_ids=ids[0:10])

print(len(dict_result))



10


In [11]:
dict_result_2 = ler_sheets_por_ids_com_esteroides(lista_ids=ids[0:10], max_workers=10)

print(len(dict_result_2))


10
